In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import os
import pickle
import numpy as np
import json
from pathlib import Path

np.set_printoptions(precision=3, suppress=True)
# uncomment and replace with correct path if there are import errors
# sys.path.append("/path/to/surface-sampling/")
# sys.path.append("/path/to/NeuralForceField/")
# os.environ["LAMMPS_POTENTIALS"] = "/path/to/lammps/potentials/"
# os.environ["LAMMPS_COMMAND"] ="/path/to/lammps/src/lmp_serial"
# os.environ["ASE_LAMMPSRUN_COMMAND"] = os.environ["LAMMPS_COMMAND"]

import ase

from mcmc.system import SurfaceSystem
from mcmc.utils.misc import get_atoms_batch
import torch
from nff.io.ase_calcs import NeuralFF
from mcmc.calculators import EnsembleNFFSurface

Initialize test slab and parameters

In [13]:
# Load prepared pristine slab
element = []
slab_pkl = open("../tutorials/data/SrTiO3_001_2x2_pristine_slab.pkl", "rb")
slab = pickle.load(slab_pkl)
offset_data_path = os.path.join(
    "../tutorials",
    "data/nff",
    "offset_data.json",
)

ref_slab_files = [
    "data/SrTiO3_001/O44Sr12Ti16.cif",
    "data/SrTiO3_001/O36Sr12Ti12.cif",
    "data/SrTiO3_001/O40Sr16Ti12.cif",
]

ref_slabs = [ase.io.read(f) for f in ref_slab_files]

In [14]:
# Initialize paths
surface_name = "SrTiO3(001)"
run_folder = Path() / surface_name
run_folder.mkdir(parents=True, exist_ok=True)

calc_settings = {
    "calc_name": "NFF",
    "optimizer": "BFGS",
    "chem_pots": {"Sr": -2, "Ti": 0, "O": 0},
    "relax_atoms": True,
    "relax_steps": 20,
    "offset": True,
    "offset_data": json.load(open(offset_data_path, "r")),
}

system_settings = {
    "surface_name": surface_name,
    "surface_depth": 1,
    "cutoff": 5.0,
    "near_reduce": 0.01,
    "planar_distance": 1.5,
    "no_obtuse_hollow": True,
}

Obtain adsorption sites

In [15]:
from pymatgen.analysis.adsorption import AdsorbateSiteFinder
from pymatgen.io.ase import AseAtomsAdaptor

pristine_slab = slab.copy()
pristine_pmg_slab = AseAtomsAdaptor.get_structure(pristine_slab)
site_finder = AdsorbateSiteFinder(pristine_pmg_slab)
# new standardized params
ads_positions = site_finder.find_adsorption_sites(
    put_inside=True,
    symm_reduce=False,
    near_reduce=system_settings["near_reduce"],
    distance=system_settings["planar_distance"],
    no_obtuse_hollow=system_settings["no_obtuse_hollow"],
)["all"]

print("adsorption coordinates are")
print(ads_positions)

adsorption coordinates are
[array([ 7.871,  7.941, 18.82 ]), array([ 1.968,  1.951, 18.779]), array([ 1.968,  0.146, 18.732]), array([ 7.871,  3.956, 18.82 ]), array([ 1.968,  5.936, 18.779]), array([ 1.968,  4.131, 18.732]), array([ 3.936,  7.941, 18.82 ]), array([ 5.903,  1.951, 18.779]), array([ 5.903,  0.146, 18.732]), array([ 3.936,  3.956, 18.82 ]), array([ 5.903,  5.936, 18.779]), array([ 5.903,  4.131, 18.732]), array([ 1.968,  7.026, 18.756]), array([ 0.984,  6.938, 18.8  ]), array([ 5.903,  7.026, 18.756]), array([ 6.887,  6.938, 18.8  ]), array([ 6.887,  0.058, 18.776]), array([ 7.871,  5.936, 18.779]), array([ 6.887,  4.946, 18.8  ]), array([ 0.984,  4.946, 18.8  ]), array([ 2.952,  6.938, 18.8  ]), array([ 6.887,  0.961, 18.8  ]), array([ 0.984,  0.961, 18.8  ]), array([ 7.871,  1.951, 18.779]), array([ 6.887,  4.043, 18.776]), array([ 5.903,  5.033, 18.756]), array([ 4.919,  4.946, 18.8  ]), array([ 4.919,  4.043, 18.776]), array([ 4.919,  0.058, 18.776]), array([ 4.919, 

Set up NFF (calculator). We are using neural network weights from our Zenodo dataset (https://zenodo.org/record/7927039). The ensemble requires an `offset_data.json` file

In [16]:
if torch.cuda.is_available():
    DEVICE = 0
else:
    DEVICE = "cpu"

# requires an ensemble of models in this path and an `offset_data.json` file
nnids = ["model01", "model02", "model03"]
model_dirs = [
    os.path.join(
        "../tutorials",
        "data/nff",
        str(x),
        "best_model",
    )
    for x in nnids
]

models = []
for modeldir in model_dirs:
    m = NeuralFF.from_file(modeldir, device=DEVICE).model
    models.append(m)

nff_surf_calc = EnsembleNFFSurface(models, device=DEVICE)
nff_surf_calc.set(**calc_settings)

offset data: {'bulk_energies': {'O': -0.17747231201, 'Sr': -0.06043637668, 'SrTiO3': -1.470008697358702}, 'stoidict': {'Sr': 0.49995161381315867, 'Ti': -0.0637500349111578, 'O': -0.31241304903276834, 'offset': -11.324476454433157}, 'stoics': {'Sr': 1, 'Ti': 1, 'O': 3}, 'ref_formula': 'SrTiO3', 'ref_element': 'Ti'} is set from parameters
chemical potentials: {'Sr': -2, 'Ti': 0, 'O': 0} are set from parameters
offset data: {'bulk_energies': {'O': -0.17747231201, 'Sr': -0.06043637668, 'SrTiO3': -1.470008697358702}, 'stoidict': {'Sr': 0.49995161381315867, 'Ti': -0.0637500349111578, 'O': -0.31241304903276834, 'offset': -11.324476454433157}, 'stoics': {'Sr': 1, 'Ti': 1, 'O': 3}, 'ref_formula': 'SrTiO3', 'ref_element': 'Ti'} is set from parameters


Initialize surface system

In [17]:
slab_batch = get_atoms_batch(
    slab,
    system_settings["cutoff"],
    DEVICE,
    props={"energy": 0, "energy_grad": []},
)

surface = SurfaceSystem(
    slab_batch,
    ads_positions,  # TODO: move ads_positions to be created within SurfaceSystem
    calc=nff_surf_calc,
    system_settings=system_settings,
    default_io_path=run_folder,
)
surface.all_atoms.write(run_folder / "SrTiO3_001_2x2_all_virtual_ads.cif")

2024-06-28 16:47:35,440|INFO|initializing 64 virtual atoms
2024-06-28 16:47:35,445|INFO|Initial state is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2024-06-28 16:47:35,446|INFO|Number of pristine atoms is 60
2024-06-28 16:47:35,454|INFO|bulk indices are [ 0  1  2  3  4  5  6  9 10 11 12 13 14 15 16 17 18 19 20 21 24 25 26 27
 28 29 30 31 32 33 34 35 36 39 40 41 42 43 44 45 46 47 48 49 50 51 54 55
 56 57 58 59]
2024-06-28 16:47:35,455|INFO|surface indices are [ 7  8 22 23 37 38 52 53]
2024-06-28 16:47:35,456|INFO|constraints are FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 54, 55, 56, 57, 58, 59])


      Step     Time          Energy         fmax
BFGS:    0 16:47:35     -467.521881        0.2044
BFGS:    1 16:47:35     -467.525757        0.1803
BFGS:    2 16:47:35     -467.540619        0.0793
BFGS:    3 16:47:35     -467.540924        0.0598
BFGS:    4 16:47:36     -467.541351        0.0059


In [18]:
surface.get_surface_energy(recalculate=True)

      Step     Time          Energy         fmax
BFGS:    0 16:47:36     -467.521881        0.2044
BFGS:    1 16:47:36     -467.525757        0.1803
BFGS:    2 16:47:36     -467.540619        0.0793
BFGS:    3 16:47:36     -467.540924        0.0598
BFGS:    4 16:47:36     -467.541351        0.0059


array([12.471], dtype=float32)

Test out other reference slabs

In [19]:
ref_slab_batches = [
    get_atoms_batch(
        slab,
        system_settings["cutoff"],
        DEVICE,
        props={"energy": 0, "energy_grad": []},
    )
    for slab in ref_slabs
]

ref_surfs = []
for ref_slab_batch in ref_slab_batches:
    ref_surf = SurfaceSystem(
        ref_slab_batch,
        ads_positions,
        calc=nff_surf_calc,
        system_settings=system_settings,
        default_io_path=run_folder,
    )
    ref_surfs.append(ref_surf)

2024-06-28 16:47:36,869|INFO|initializing 64 virtual atoms
2024-06-28 16:47:36,874|INFO|Initial state is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2024-06-28 16:47:36,875|INFO|Number of pristine atoms is 72
2024-06-28 16:47:36,883|INFO|bulk indices are [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 66 67 68 69 70 71]
2024-06-28 16:47:36,884|INFO|surface indices are [64 65]
2024-06-28 16:47:36,884|INFO|constraints are FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 66, 67, 68, 69, 70, 71])


      Step     Time          Energy         fmax
BFGS:    0 16:47:37     -570.127991        0.7374
BFGS:    1 16:47:37     -570.142517        0.6409
BFGS:    2 16:47:37     -570.188354        0.1144
BFGS:    3 16:47:37     -570.188721        0.1004
BFGS:    4 16:47:37     -570.189758        0.0113
BFGS:    5 16:47:37     -570.189758        0.0097


2024-06-28 16:47:37,791|INFO|initializing 64 virtual atoms
2024-06-28 16:47:37,796|INFO|Initial state is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2024-06-28 16:47:37,797|INFO|Number of pristine atoms is 60
2024-06-28 16:47:37,806|INFO|bulk indices are [ 0  1  2  3  4  5  6  8  9 10 11 12 13 14 15 16 17 18 19 20 21 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 38 39 40 41 42 43 44 45 46 47 48 49 50
 51 53 54 55 56 57 58 59]
2024-06-28 16:47:37,814|INFO|surface indices are [ 7 22 37 52]
2024-06-28 16:47:37,820|INFO|constraints are FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58, 59])


      Step     Time          Energy         fmax
BFGS:    0 16:47:38     -467.525543        0.1416
BFGS:    1 16:47:38     -467.526703        0.1320
BFGS:    2 16:47:38     -467.534088        0.0035


2024-06-28 16:47:38,390|INFO|initializing 64 virtual atoms
2024-06-28 16:47:38,395|INFO|Initial state is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2024-06-28 16:47:38,396|INFO|Number of pristine atoms is 68
2024-06-28 16:47:38,409|INFO|bulk indices are [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 62 65 66 67]
2024-06-28 16:47:38,410|INFO|surface indices are [60 61 63 64]
2024-06-28 16:47:38,412|INFO|constraints are FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 62, 65, 66, 67])


      Step     Time          Energy         fmax
BFGS:    0 16:47:38     -518.694092        0.7792
BFGS:    1 16:47:38     -518.722046        0.6676
BFGS:    2 16:47:38     -518.753479        1.1361
BFGS:    3 16:47:38     -518.766785        0.3977
BFGS:    4 16:47:39     -518.778076        0.2571
BFGS:    5 16:47:39     -518.780762        0.2640
BFGS:    6 16:47:39     -518.782776        0.0935
BFGS:    7 16:47:39     -518.783081        0.0494
BFGS:    8 16:47:39     -518.783203        0.0442
BFGS:    9 16:47:39     -518.783386        0.0503
BFGS:   10 16:47:39     -518.783447        0.0373
BFGS:   11 16:47:40     -518.783508        0.0181
BFGS:   12 16:47:40     -518.783508        0.0108
BFGS:   13 16:47:40     -518.783630        0.0085


In [20]:
for surf in ref_surfs:
    surf_energy = surf.get_surface_energy()
    print(f"energy of reference slab is {surf_energy}")

energy of reference slab is [35.931]
energy of reference slab is [12.478]
energy of reference slab is [-4.876]
